# Session 3

## Time series, graphs and spatial data

During this final session we will look at some practical hydrological applications of the various Python libraries. There is only so much we can cover during a single session but this will give you a flavour of what's possible. 

Before doing anything, we must import the libraries we'll be using

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Time series statistics

### Stream discharge

We ended the previous session with fitting a power equation to the stage-discharge data for Elkhead creek. We'll continue by converting the measured stream stages to discharge values. From the time series obtained that way, we can calculate some descriptive statistics.

Like we learned last session, we can load the data and parse the dates in the file using `Pandas`' `read_csv` function

In [ ]:
# USGS 09246500 ELKHEAD CREEK NEAR CRAIG, CO
# https://waterdata.usgs.gov/nwis/measurements/?site_no=09246500
!wget https://raw.githubusercontent.com/vincentpost/PythonEssentialsV/main/09246500_time_series_data.txt
df = pd.read_csv('09246500_time_series_data.txt', 
                 sep='\t',
                 index_col=2,
                 parse_dates=True,
                 skiprows=[i for i in range(32) if i != 30])
df.head()

The file header explains that the column named '279703_00065' contains the gauge heights, and the column '212023_00060' has the discharge (in cubic feet per second, CFS). Printing the first five rows of data shows there are lots of NaN values. Closer inspection of the file (in a text editor or a spreadsheet program) reveals that most of the stage data fields are empty. The year 2021 has a nearly complete set of measurements though, so let's select that for the remainder of the exercise. This is easily done using the `year` attribute of the date/time field of the index

In [ ]:
df = df.loc[df.index.year == 2022]
df.head()

Let's quickly plot the time series

In [ ]:
df['279703_00065'].plot()

Using the rating curve from session 2 ($Q = 0.547  h ^ {5.15}$) we can convert the stream stage to a discharge. We'll add it to the dataframe `df` and give it column label Q

In [ ]:
# From session 2:
b, a = 5.1528685333467035, 0.5467695289254147
df['Q'] = a * df['279703_00065'] ** b

Let's plot the calculated discharge and compare it to the reported discharge from the downloaded file

In [ ]:
plt.figure()
plt.plot(df.index, df['Q'], '-', label='Session 2 rating curve')
plt.plot(df.index, df['212023_00060'], '-', label='USGS reported value')
plt.legend()
plt.ylabel('Discharge [CFS]');

***Exercise 3.1***: Create a scatter plot that shows the USGS reported flow rates on the horizontal axis and the flow rates according to our rating curve on the vertical axis.

In [ ]:
# Type your code here


The flow rates don't exactly line up, but since this is an exercise only, let's continue with our calculated discharge data. If this were a real investigation, we'd be scrutinising the data a lot more of course and try to understand the causes for these differences. Note for example that the file hardly contains any discharge figures for the winter season (even though the stream stage was measured), presumably because the stream was frozen. The results of what follows therefore make no hydrological sense, but the purpose is not to analyse this stream, the purpose is to demonstrate the usage of `Pandas` to calculate the probabilities of exceedance. 

The first step in the process is to resample the 15-minute data to daily values

In [ ]:
dfd = df['Q'].resample('1D').mean()
dfd.head()

Because there is only one column of data, `Pandas` actualy just created a `Series` object instead of a `DataFrame`. Although they are similar, they are different at the same time. For example, both a `Series` and a `DataFrame` have an index, unlike a `DataFrame`, a `Series` does not have column names.

***Exercise 3.2***: Use the `Pandas` function `describe` to calculate the descriptive statistics of the daily values (note that you can also use individual functions like `min`, `max`, `mean`, `median` and `std`)

In [ ]:
# Type your code here


***Exercise 3.3***: Look up the syntax for `matplotlib`'s `hist` function to plot a histogram of the data. For an explanation of this function, refer to <A href="https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html">https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html</A>.

In [ ]:
# Type your code here


Because there is only one column of data, `Pandas` actualy just created a `Series` object instead of a `DataFrame`. Although they are similar, they are different at the same time. For example, both a `Series` and a `DataFrame` have an index, unlike a `DataFrame`, a `Series` does not have column names. Also, we can't easily add a column to `dfd` like we did in the above example for the discharge Q. To overcome this, we can simply use `to_frame` to convert the `Series` back into a `DataFrame`. 

In [ ]:
dfd = dfd.to_frame()
dfd.head()

The `rank` method in `Pandas` computes the numerical data ranks. If there are duplicate values then equal values are assigned a rank that is the average of the ranks of those values. Using `pct=True` converts the ranks into percentiles. For plotting purposes, the data need to be sorted, which is done using the `sort_values` function. The first argument determines which column is used for sorting, in this case it is 'cdf'.

In [ ]:
dfd['cdf'] = dfd.rank(method='average', pct=True)
dfd = dfd.sort_values('cdf', ascending=False)
dfd.head()

Plotting the cumulative density function is easy. For flow duration curves it is common to show the discharge values in order of decreasing probability, which is accomplished by using the `xlim` function to reverse the min and max values of the $x$ axis.

In [ ]:
plt.figure()
plt.semilogy(dfd['cdf'] * 100, dfd['Q'])
plt.xlim(100, 0)
plt.xlabel('Exceedance probability [%]')
plt.ylabel('Discharge [CFS]')
plt.grid(True, which="both", ls=':', color='0.65')
#plt.savefig('cdf.png', dpi=300)

Note that there are different ways to calculate a flow duration curve. This is just one illustrative example. More sophisticated methods can also be implemented, but that is beyond an essentials course.

## More on graphs

You may have seen them: The warming stripes that show how annual temperatures have increased over the past few decades. They were first created by  Ed Hawkings (see <A href="https://www.climate-lab-book.ac.uk/2018/warming-stripes/">Climate Lab Book</A>) and were adopted as the logo of the <A href="https://scientistsforfuture.org/">Scientists for Future</A> movement. The `Matplotlib` homepage contains an <A href="https://matplotlib.org/matplotblog/posts/warming-stripes/">explanation</A> on how to create them, but it is a rather complex example and there is in fact a much easier way, which is included as a bonus exercise at the end of this notebook. 

For the next exercise we will be using the HadCRUT5 near surface temperature data set to demonstrate some of `Matplotlib`'s capabilities. The aim is to plot three graphs in a single figure and add an uncertainty band around the data. The HadCRUT5 data set is created by the British Met Office Hadley Centre. The files for this exercise were downloaded on 19 April 2022 from <A href="https://www.metoffice.gov.uk/hadobs/hadcrut5/data/current/download.html">https://www.metoffice.gov.uk/hadobs/hadcrut5/data/current/download.html</A>. They contain the annual mean temperature anomalies for the northern- and southern hemisphere, as well as the entire globe since 1850. The temperature anomaly is the difference of the annual temperature and the mean annual temperature for the reference period 1961 - 1990.

Let's have a look at the data. First, we'll store the filenames in a `list`, so that we can automate the creation of the graphs with a `for` loop later on

In [ ]:
# https://www.metoffice.gov.uk/hadobs/hadcrut5/data/current/download.html
!wget https://raw.githubusercontent.com/vincentpost/PythonEssentialsV/main/HadCRUT.5.0.1.0.analysis.summary_series.global.annual.csv
!wget https://raw.githubusercontent.com/vincentpost/PythonEssentialsV/main/HadCRUT.5.0.1.0.analysis.summary_series.northern_hemisphere.annual.csv
!wget https://raw.githubusercontent.com/vincentpost/PythonEssentialsV/main/HadCRUT.5.0.1.0.analysis.summary_series.southern_hemisphere.annual.csv
fnames = [
    'HadCRUT.5.0.1.0.analysis.summary_series.global.annual.csv',
    'HadCRUT.5.0.1.0.analysis.summary_series.northern_hemisphere.annual.csv',
    'HadCRUT.5.0.1.0.analysis.summary_series.southern_hemisphere.annual.csv',
]

Also, later on we'll want a title above each graph, so let's store those in a `list` as well (making sure that the order of the titles is consistent with the order in `fnames`)

In [ ]:
titles = ['Global', 'Northern hemisphere', 'Southern hemisphere']

As there are three sets of data, we'll need three individual graphs. For this, we'll use `Matplotlib`'s `subplots` function.

The `for` loop executes a number of steps. First, it uses `Pandas`' `read_csv` function to read the data files. It then uses `Matplotlib`'s `plot`function to plot the temperature anomaly as a function of time. Because the data also contain information about the uncertainty, we can plot an envelope around the line to indicate the uncertainty range. Note that the envelope is plotted in black (hence the 'k' for the color) but with a transparency `alpha` of 20% to make it appear grey. Each graph is also give a label on the y axis and a title. The final command (`plt.tight_layout()`) optimizes the layout of the figure.

In [ ]:
fig, axs = plt.subplots(nrows=3, figsize=(7,6))

for fn, title, ax in zip(fnames, titles, axs):
    df = pd.read_csv(fn, index_col=0)
    
    ax.plot(df.index, df['Anomaly (deg C)'])
    ax.fill_between(df.index, df.iloc[:, 1], df.iloc[:, 2], color='k', alpha=0.2)

    ax.set_ylabel('Temp. anom. [\u2103]')
    ax.set_title(title)
    
plt.tight_layout()

***Exercise 3.4***: Run the code cell above but comment out the `plt.tight_layout()` line. Observe what happens.

***Exercise 3.5***: The range of the horizontal axis can be set using `ax.set_xlim(xmin, xmax)`, in which `xmin` and `xmax` are the axis limits. Add this function to the code cell above the figure to plot only the data between 1920 and 2020.

## Special functions

In the following example it will be demonstrated how to calculate the drawdown (i.e. the fall of the groundwater level) of an abstraction well as a function of time. The function that calculates the drawdown of the well as a function of time $t$ and radial distance $r$ is

$s = \frac{Q}{4 \pi T} W(u)$

where $Q$ is the well abstraction rate, $T$ the aquifer transmissivity and $W(u)$ is the Theis well function, known as the exponential integral outside hydrogeology

$\int_u^\infty \frac{e^u}{u} du$

in which 

$u = \frac{r^2S}{4Tt}$

where $S$ is the aquifer storativity.

As luck would have it, the exponential integral is part of `scipy`'s special functions, so all we have to do is import it, and we can code up a function to calculate the drawdown according to the Theis equation. You'll notice that a function is defined by typing `def` followed by the function name (`theis` in this example). The arguments that can be passed to the function are enclosed between parentheses. The arguments `r` (radial distance from the well), `t` (time), `S` (aquifer storativity) and `T` (aquifer transmissivity) are compulsory. The argument `Q` (the well discharge) is a so-called keyword argument or kwarg, which is an argument that is assigned a default value (in this case 100) in the function definition. The user does not have to specify a value for `Q` unless they want to override the default value. The drawdown is stored in array `s` and in the final line of the function, this array is 'returned' to the part of the program that called the function.

In [ ]:
from scipy.special import exp1

def theis(r, t, S, T, Q=100): # Q is a kwarg (keyword argument), it is optional and has a default value
    u = S * r ** 2 / (4.0 * T * t)
    s = Q / (4 * np.pi * T) * exp1(u)
    return s

With the function defined, we can create a graph that shows the drawdown for a given set of aquifer parameters after 0.1, 1, and 10 days since the well started pumping

In [ ]:
fig, ax = plt.subplots()
S = .01 # Storativity
T = 10 # Transmissivity
r = np.linspace(0.1, 100, 200) # Radial distance from well
for t in [0.1, 1, 10]: # Times
    ax.plot(r, theis(r, t, S, T), label=f'time = {t} d')

ax.set_xlabel('radial distance (m)')
ax.set_ylabel('drawdown (m) ')
ax.legend(loc='best')

## Geospatial data

Before you can run the next code cell, you must have installed `pyshp`. Once `pyshp` is installed you can import it, oddly enough not by specifying the package name but by:

In [ ]:
import shapefile

The concents of the shapefile can be read by creating an instance of the `Reader` class. In the following code cell the shapefile `river_vertices.shp` is imported and the geometry type is displayed on the screen

In [ ]:
sf = shapefile.Reader("shp/river_vertices.shp")
sf.shapeTypeName

Once the file is loaded, we can loop over all the river nodes, and get their coordinates using the `Reader`'s `shapes` function. Note that the coordinates of each point are stored as a list within a list. The `oid` attribute is the object identifier, which defines the point's position in the original shapefile.

In [ ]:
for pt in sf.shapes():
    print(pt.oid, pt.points)

It is also possible to loop through the geometries and fields simultaneously. This is done with the `shapeRecords` function. There is only one field in the shapefile, so the `record` attribute is a list with only one item per geometry.

In [ ]:
for v in sf.shapeRecords():
    print(v.shape.points[0], v.record)

***Exercise 3.6***: Rewrite the for loop above so that the coordinates of the points are displayed in a nicer fashion (for example, something like: Point 1: x = 639435.74, y = 5791486.69)

In [ ]:
# Type your code here


Now let's use the coordinate data to calculate the distance of each point measured to its upstream point. First we use list comprehension to store separate arrays of the points' x and y values

In [ ]:
xvalues = np.array([pt.points[0][0] for pt in sf.shapes()])
yvalues = np.array([pt.points[0][1] for pt in sf.shapes()])
print(yvalues)

Using NumPy's `diff` function, we can calculate the difference between the distance in the x and y direction between each point.

In [ ]:
dx = np.diff(xvalues)
dy = np.diff(yvalues)
print(dx)

Calculating the distance between each point simply involves applying the Pythagoras theorem. The total distance along the river course measured from the most upstream point is obtained by summing all the distances between the consecutive pairs of points

In [ ]:
l = np.sqrt(dx ** 2 + dy ** 2)
lsum = np.cumsum(l)
print(lsum)

With the distance of each point known, `numpy`'s `interp` function can be used to do the interpolation

In [ ]:
l0 = 0
l1 = lsum[-1]
stage0 = sf.record(0)[0]
stage1 = sf.record(-1)[0]
lint = lsum[:-1]

stageint = np.interp(lint, [l0, l1], [stage0, stage1])
stageint = np.hstack((stage0, stageint, stage1))

Finally, the interpolated values are saved as an ESRI Shapefile

In [ ]:
w = shapefile.Writer('shp/river_vertices_int.shp')
w.field('stageint', 'F', decimal=4)
for pt, s in zip(sf.shapes(), stageint):
    w.point(pt.points[0][0], pt.points[0][1])
    w.record(stageint=s)
w.close()

## Bonus material: Climate stripes

This next example will use the HeadCRUT5 data set that we visualized earlier to recreate the famouse climate stripes (see <A href="https://www.climate-lab-book.ac.uk/2018/warming-stripes/">Climate Lab Book</A>). The graph will look slightly different, but definitively close enough. You will notice how compact the code is, demonstrating the power of Python to get rather complex tasks done with a minimal amount of code.

Any temperature data set could be used for this purpose, so you may want to try your own after completing this exercise.

Now let's use the global temperature data set to create the warming stripes. We'll need the first filename from the list and we're only interested in the anomalies, so we'll just store this column in a DataFrame

In [ ]:
df = pd.read_csv(fnames[0], index_col=0)['Anomaly (deg C)']
df.head()

Note once more that `df` is not a DataFrame but a Series. For the purposes of this exercise this does not matter though.

To control the colour of the warming stripes, we will use one of `Matplotlib`'s colormaps (see <A href="https://matplotlib.org/stable/tutorials/colors/colormaps.html">https://matplotlib.org/stable/tutorials/colors/colormaps.html</A> for more information). The one that gives the closest resemblance to the original warming stripes is the colormap called bwr (short for blue, white, red). In order to be able to use it, we must first import the `cm` package from `Matplotlib`

In [ ]:
from matplotlib import cm

cm.bwr(0)

The first line does the importing, the second line calls the `bwr` colormap with 0 as a function argument. The return value is a tuple with four values. The first three are the RGB (red, green and blue) values, the fourth is the alpha value, which we have seen in the example above. Since the color blue is on left side of the colormap, it makes sense that the RGB value is 1.0 (i.e., 100%) for blue. The representation of colours in this way is called the RGBA model.

***Exercise 3.8***: Execute the code cell below. Does the result make sense?

In [ ]:
print(cm.bwr(0.5))
print(cm.bwr(1.0))

So how can we use this to create the warming stripes? The first step is to scale all temperature anomalies between 0 and 1. Then we can use `cm.bwr` to get the corresponding colours from the colormap. This is where list comprehension comes in handy again: For each year we can store the colour in a list by looping over all values, scale it between 0 and 1 and convert it to a RGBA colour representation.

In [ ]:
ymin = df.min()
ymax = df.max()
colors =  [cm.bwr((y - ymin) / (ymax - ymin)) for y in df]

The final step is to create a bar graph of the data using `Matplotib`'s `bar` function. Note that all bars have a height of 1 and a widht of 1 (year). The color of each bar is passed using the `color` argument of the `bar` function.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 1))
ax.bar(df.index, np.ones(len(df)), width=1, color=colors)
ax.axis('off')

***Exercise 3.9***: Create the same warming stripes but use another colormap. You can find their names on <A href="https://matplotlib.org/stable/tutorials/colors/colormaps.html">https://matplotlib.org/stable/tutorials/colors/colormaps.html</A>.

In [ ]:
# Type your code here